In [1]:
from pyudptest import Device, CommandPacket
from matplotlib import pyplot as plt
import asyncio
from ipywidgets import interact, interact_manual, Output, VBox, HTML, widgets
import nest_asyncio
from commands import *
nest_asyncio.apply() # For being able to use asyncio.run()
import time


# FAT test "Medusa"

Welcome to the FAT-test for the medusa instrument. The tests in this program ensure that the Medusa system is properly installed. 

The first tests test that all components are installed correctly. If a stand-alone test needs to be done on the pump module it can be made with the "Pump module test". If the complete system is suposed to be tested the test can be skipped and proceed with the system test.

# Set IP adress

Set the IP adress for the device that shall be tested

In [2]:
ip = "192.168.1.55"

@interact(ip=ip)
def set_ip(ip):
    ip = ip

interactive(children=(Text(value='192.168.1.55', description='ip'), Output()), _dom_classes=('widget-interact'…

# Pump module test.


The test can be run on either a free-standing pump or on a pump installed in the instrument. 
The test is preferably run on a free-standing pump prior to installation of the pump to make sure that the pump is working and again after installation to make sure that the installation was sucessfull.

Note: If only one pump is connected to the valve board it must be in "Pump 1" 

The test connects to each of the components included in the pump module and return "ok" if its working and "fail" if not. 

In [18]:
async def test_pump(Unit):
    # Check connection with PCB by checking software number
    number = await get_software_number(device_ip="192.168.1.55", module=Unit)
    print(number)

    # Check pressure sensors
    for x in range(1, 5):
        pressure = await get_pressure(device_ip="192.168.1.55", pos=x, unit=Unit)  # Correct 'ip' parameter
        if pressure != 40:
            print("Pressure sensor", x, "ok")
        else:
            print("Error in sensor", x)
    #Check motor statu:
    #...........

    #Check homing sensor status. 
    #.......

    # Check Homing of pump
    home = await home_pump(device_ip=ip , flow_rate=0.001, module =Unit)  # Correct 'ip' parameter
    if home == '0':
        print("Home OK")
    else:
        print("Home failed")

    # Check valves
    valves = await set_nitro_valves(device_ip=ip, state= 1, pump_unit= Unit)  # Correct 'ip' parameter
    if valves == '0':
        print("Nitrogen valves ok")
    else:
        print("Error in valves")
    await set_nitro_valves(device_ip=ip, state= 0, module= Unit)

@interact_manual(Unit=[('Pump 1 (Left)', "pump1"), ('Pump 2 (Right)', "pump2")])
def select_method(Unit):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(test_pump(Unit))  # Use 'run_until_complete' to run the async function

interactive(children=(Dropdown(description='Unit', options=(('Pump 1 (Left)', 'pump1'), ('Pump 2 (Right)', 'pu…

# System connections 

In [13]:
async def test_system_connections():
    # Test pumps
    await test_pump("pump1")
    await test_pump("pump2")

    # Test valve board
    number = await get_software_number(device_ip=ip, module="valve")
    print("Valve Software Number:", number)

    # Check valves
    valves = ["pump1", "pump2", "solvent"]
    
    for valve in valves:
        home = await home_valve(device_ip=ip, valve_type=valve)
        if home[0] == '0':
            print(valve, "OK")
        else:
            print(valve, "Failed")
    
    home = await home_drawer(device_ip=ip)
    if home == '0':
        print("Tray OK")
    else:
        print("Tray Failed")

    # Check LED
    led = await set_led(device_ip=ip, data1=100, data2=100, data3=100, data4=100)
    if led.data[0] == '0':
        print("LED OK")
    else:
        print("LED Failed")
    await set_led(device_ip=ip, data1=0, data2=0, data3=0, data4=0)

    # check pressure sensor 
    press = await set_pressure_regulator(device_ip=ip, pressure=0)
    if press == '0':
        print("Pressure regulator OK")
    else:
        print("Pressure regulator failed") 

    #check fan
    fan= await set_fan(device_ip=ip, speed=100)
    if fan == '0':
        print("Fan OK")
    else:
        print("Fan Failed")
    fan= await set_fan(device_ip=ip, speed=0) 

    #check solvent valves.
    solvalve= await set_solvent_valves(device_ip=ip, state=1)
    if solvalve == '0':
        print("solvent valves OK")
    else:
        print("solvent valves Failed") 
 
@interact_manual()
def select_method():
    loop = asyncio.get_event_loop()
    loop.create_task(test_system_connections())


interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

# Correct flow channels

# Leakage test

The leakage test tests that all there are no leakages in the system. 

In [12]:
# home the pumps and valves 
async def leakage_test(Side):

    await home_pump(device_ip="192.168.1.55", flow_rate=0.001, module=Side )
    await home_valves(device_ip="192.168.1.55")
        
    # Set the pressure regulator to max voltage
    await set_pressure_regulator(device_ip="192.168.1.55", pressure= 5)
        
    channel_values = {"sample": 0, "column": 15, "solvent": 14, "rinse": 13}
    channels = ["sample", "column", "solvent", "rinse"]

    for channel in channels:
        value = channel_values.get(channel)
        
        # Vänta på att ventilpositionen ställer in ventilen till sample positionen. 
        await set_valve_pos(device_ip="192.168.1.55", valve_type="pump2", pos=value)
        
        # Sätt nitrogenventiler till öppet läge
        await set_nitro_valves(device_ip="192.168.1.55", state=1)
        await asyncio.sleep(1)
        
        # Läs av tryckvärderna
        pressure1_start = await get_pressure(device_ip="192.168.1.55", pos=1)
        pressure2_start = await get_pressure(device_ip="192.168.1.55", pos=2)
        pressure3_start = await get_pressure(device_ip="192.168.1.55", pos=3)
        pressure4_start = await get_pressure(device_ip="192.168.1.55", pos=4)

        # Stäng av ventilerna och vänta på trycket i 5 sekunder
        await set_nitro_valves(device_ip="192.168.1.55", state=0)
        await asyncio.sleep(5)
        
        pressure1 = await get_pressure(device_ip="192.168.1.55", pos=1)
        pressure2 = await get_pressure(device_ip="192.168.1.55", pos=2)
        pressure3 = await get_pressure(device_ip="192.168.1.55", pos=3)
        pressure4 = await get_pressure(device_ip="192.168.1.55", pos=4)

        if pressure1 < pressure1_start*0.5 or pressure2 < pressure2_start*0.5 or pressure3 < pressure3_start*0.5 or pressure4 < pressure4_start*0.5:
            print("Leakage in ", channel)
        else:
            print(channel, " Approved")

    # Set position to solvent output to test the solvent valve
    await set_valve_pos(device_ip="192.168.1.55", valve_type="pump2", pos=14)

    for x in range(8):
        await set_valve_pos(device_ip="192.168.1.55", valve_type="solvent", pos=x)
            
        # Sätt nitrogenventiler till öppet läge
        await set_nitro_valves(device_ip="192.168.1.55", state=1)
        await asyncio.sleep(1)
        
        # Läs av tryckvärderna
        pressure1_start = await get_pressure(device_ip="192.168.1.55", pos=1)
        pressure2_start = await get_pressure(device_ip="192.168.1.55", pos=2)
        pressure3_start = await get_pressure(device_ip="192.168.1.55", pos=3)
        pressure4_start = await get_pressure(device_ip="192.168.1.55", pos=4)

        # Stäng av ventilerna och vänta på trycket i 5 sekunder
        await set_nitro_valves(device_ip="192.168.1.55", state=0)
        await asyncio.sleep(5)
        
        pressure1 = await get_pressure(device_ip="192.168.1.55", pos=1)
        pressure2 = await get_pressure(device_ip="192.168.1.55", pos=2)
        pressure3 = await get_pressure(device_ip="192.168.1.55", pos=3)
        pressure4 = await get_pressure(device_ip="192.168.1.55", pos=4)
        
        if pressure1 < pressure1_start*0.5 or pressure2 < pressure2_start*0.5 or pressure3 < pressure3_start*0.5 or pressure4 < pressure4_start*0.5:
            print("Leakage in ", x)
        else:
            print("Solvent line ", x , " Approved")

@interact_manual(Side=[('Left', "pump1 "), ('Right', "pump2")])
def select_method(Side):
    loop = asyncio.get_event_loop()
    loop.create_task(leakage_test(Side))

interactive(children=(Dropdown(description='Side', options=(('Left', 'pump1 '), ('Right', 'pump2')), value='pu…

# Tube set-up

Tests that the channels are correctly installed; All tubes are in the specified inlet and outlet.

The test have a manual interaction and uses 4 container with water to visulize the correct flow path.
When checking pumps it starts with checking channel 1- sample line. Attach external tubing to the sample ports and when ready the pressurized air is creating bubbles in the container if correct installed. 
Then it continus to check the channel 2- column line, channel 3- solvent line and channel 4 rinsing line. 


In [7]:
async def channel_test(module, channel):
    await set_valve_pos(device_ip=ip, valve_type=module, pos=0)
    await set_pressure_regulator(device_ip=ip, pressure= 5)



    if module == "pump1" or module == "pump2":
        
        if channel == "Sample":
            await set_valve_pos(device_ip=ip, valve_type=module, pos=0)
            await set_nitro_valves(device_ip=ip, state= 1, pump_unit = module)
            await asyncio.sleep(10)
            await set_nitro_valves(device_ip=ip, state= 0, pump_unit = module)

        if channel == "Column":
            await set_valve_pos(device_ip=ip, valve_type=module, pos=15)
            await set_nitro_valves(device_ip=ip, bank= module, state=1)
            await asyncio.sleep(10)
            await set_nitro_valves(device_ip=ip, state= 0, pump_unit = module)
        
        if channel == "solvent":
            await set_valve_pos(device_ip=ip, valve_type=module, pos=14)
            await set_solvent_valves(device_ip=ip,bank=1, state=1)

            for x in range(8):   
                await set_valve_pos(device_ip=ip, valve_type="solvent", pos=x)
                await set_solvent_valves(device_ip=ip,bank=1, state=1)
                await set_nitro_valves(device_ip=ip, state= 1, pump_unit = "pump1")
                await asyncio.sleep(10)
                await set_nitro_valves(device_ip=ip, state= 0, pump_unit = "pump1")
                await set_solvent_valves(device_ip=ip, bank= "pump1", state=1)
                
            await set_solvent_valves(device_ip=ip, bank= module, state=1)
       
        if channel == "rinse":
            await set_valve_pos(device_ip=ip, valve_type=module, pos=13)
            await set_nitro_valves(device_ip=ip, state= 1, pump_unit = module)
            await asyncio.sleep(10)
            await set_nitro_valves(device_ip=ip, state= 0, pump_unit = module)
    
@interact_manual(Unit=[('Pump 1 (Left)', "pump1"), ('Pump 2 (Right)', "pump2"), ('Solvent', "solvent")], 
                 channel=["Sample", "Column","solvent", "rinse"] )
def select_method(Unit,channel):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(channel_test(Unit,channel))

interactive(children=(Dropdown(description='Unit', options=(('Pump 1 (Left)', 'pump1'), ('Pump 2 (Right)', 'pu…